In [85]:
import tensorflow as tf
import numpy as np 
import pandas as pd 
from data_loader import bench, squat, deadlift

In [93]:
class RepCount(tf.keras.Model):
    def __init__(self):
        super(RepCount, self).__init__()
        self.values = tf.constant([0], dtype=tf.float32)

    def heaviside_custom(self, x, values):
        ones = tf.ones_like(x)
        zeros = tf.zeros_like(x)
        return tf.where(x > 0, ones, tf.where(x < 0, zeros, values))

    def call(self, tens):
        t = self.heaviside_custom(tens, self.values)
        b = t[1:] - t[:-1]
        mask = tf.reduce_any(tf.not_equal(b, 0), axis=1)
        b = tf.boolean_mask(b, mask)
        b = tf.where(tf.equal(b, -1), tf.zeros_like(b), b)
        return tf.reduce_sum(b, axis=0)

def preprocessing(dummy_input):
    t = np.mean(dummy_input, axis=0) 
    padding = np.zeros((128 - dummy_input.shape[0] % 128, dummy_input.shape[1]))
    dummy_input = np.concatenate([dummy_input, padding], axis=0)
    dummy_input = dummy_input - t
    dummy_input_2 = np.reshape(dummy_input, (-1, 128, 3))
    dummy_input_3 = tf.constant(dummy_input_2, dtype=tf.float32)
    return dummy_input_3

model = RepCount()
dummy_input = preprocessing((bench[1][['gravityX', 'gravityY', 'gravityZ']]).dropna().to_numpy())
inp = dummy_input[0]
output = model(inp)
print("Output:", output)

Output: tf.Tensor([0. 0. 0.], shape=(3,), dtype=float32)


In [94]:
saved_model_dir = "models/model2"
tf.saved_model.save(model, saved_model_dir)


INFO:tensorflow:Assets written to: models/model2/assets


INFO:tensorflow:Assets written to: models/model2/assets


In [95]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open("rep_count_model.tflite", "wb") as f:
    f.write(tflite_model)


W0000 00:00:1741828507.013490 7847760 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1741828507.013501 7847760 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-12 18:15:07.013638: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: models/model2
2025-03-12 18:15:07.013795: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-12 18:15:07.013800: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: models/model2
2025-03-12 18:15:07.014700: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-12 18:15:07.017355: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: models/model2
2025-03-12 18:15:07.019340: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 5703 microseconds.
2025-03-12 18:15:07.031927: I tensorflow/compiler/mlir/

In [96]:
interpreter = tf.lite.Interpreter(model_path="rep_count_model.tflite")
interpreter.allocate_tensors()


input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:", input_details)
print("Output details:", output_details)

input_shape = input_details[0]['shape']

Input details: [{'name': 'serving_default_inputs:0', 'index': 0, 'shape': array([128,   3], dtype=int32), 'shape_signature': array([128,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'PartitionedCall:0', 'index': 29, 'shape': array([3], dtype=int32), 'shape_signature': array([3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [100]:
dummy_input.shape

TensorShape([10, 128, 3])

In [101]:
#interpreter.resize_tensor_input(input_details[0]['index'], dummy_input.shape)
#interpreter.allocate_tensors()  
interpreter.set_tensor(input_details[0]['index'], dummy_input[0])
interpreter.invoke()



In [102]:
output_details = interpreter.get_output_details()
output_data = interpreter.get_tensor(output_details[0]['index'])
print("Model output:", output_data)


Model output: [0. 0. 0.]
